<img src="https://www.escoladnc.com.br/wp-content/uploads/2022/06/dnc_formacao_dados_logo_principal_preto-1.svg" alt="drawing" width="300"/>

# Recomendação baseada em Co-visitation

Neste notebook iremos explorar a recomendação de itens a partir de co-visitação de itens. Este tipo de recomendação é utilizada em ofertas como "_Clientes que compraram X também compraram Y_". Em particular, neste notebook utilizaremos o dataset do [_Pinterest_](https://br.pinterest.com/) para criar ofertas como "_Quem viu X também viu Y_"

Para criar o algoritmo utilizaremos a biblioteca [NetworkX](https://networkx.org/) que possibilita a criação, manipulação e estudos de estruturas em grafos.

**Nota**: Para instalar a biblioteca `NetworkX` descomente a linha abaixo e execute a célula.

In [2]:
# !pip install networkx

In [3]:
import os
import pandas as pd
from google.colab import files
from collections import Counter
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib
from cycler import cycler

matplotlib.rcParams['axes.prop_cycle'] = cycler(color=['#007efd', '#FFC000', '#303030'])

# Carregando o dataset

_Pinterest_ é uma plataforma online onde usuários podem explorar imagens e vídeos sobre temas de sua preferência, podendo salvar imagens em seus perfis. Assim, o dataset do _Pinterest_ é composto pelos seguintes dados:

- `user_id`: identificador do usuário
- `item_id`: identificador do item (imagem ou vídeo)
- `rating`: sinal indicando que o usuário salvou o item

Para facilitar as análises, os identificador dos usuários e itens foram concatenados com a _string_ `U-` e `I-`, respectivamente.

Upload file `pinterest.parquet`

In [4]:
%%time
_ = files.upload() # approx: 30s

Saving pinterest.parquet to pinterest.parquet
CPU times: user 860 ms, sys: 95.5 ms, total: 955 ms
Wall time: 1min 4s


In [9]:
filepath = './pinterest.parquet'

df = pd.read_parquet(filepath)
df.tail()

,user_id,item_id,rating
1445617,U-55186,I-5448,1
1445618,U-55186,I-4615,1
1445619,U-55186,I-5346,1
1445620,U-55186,I-2803,1
1445621,U-55186,I-4207,1


In [10]:
n_users = df['user_id'].nunique()
n_items = df['item_id'].nunique()
n_ratings = df[['user_id', 'item_id']].drop_duplicates().shape[0]

print (f'#Usuários: {n_users}')
print (f'#Itens: {n_items}')
print (f'#Avaliações: {n_ratings}')

#Usuários: 55187
#Itens: 9911
#Avaliações: 1408394


# Criando o grafo de usuários e itens

Criaremos um grafo bipartido com as seguintes características:

- nós do tipo `item` e do tipo `usuário`
- Arestas entre um usuário e todos os itens que ele salvou

<img src="https://www.researchgate.net/publication/349424329/figure/fig3/AS:992764531585030@1613704917811/User-Item-bipartite-graph.jpg" alt="graph" width="200"/>


In [15]:
G = nx.Graph()
G.add_nodes_from(df['item_id'].unique(), node_type='item')
G.add_nodes_from(df['user_id'].unique(), node_type='user')
G.add_weighted_edges_from(df[['user_id', 'item_id', 'rating']].values) # [[no_1, no_2, peso]]

In [16]:
G

Uma vez criado o grafo, podemos analisar a quantidade de nós e arestas criados.

In [17]:
# Number of nodes
G.number_of_nodes()

65098

In [18]:
# Number of edges
G.number_of_edges()

1408394

# Construindo a recomendação a partir do grafo

Uma vez criado o grafo, podemos buscar os vizinhos de um nó utilizando a função `graph.neighbors()`

In [19]:
item_id = df['item_id'][0]
neighbors = G.neighbors(item_id)
list(neighbors)[:5]

['U-0', 'U-56', 'U-347', 'U-365', 'U-454']

In [20]:
user_id = df['user_id'][0]
neighbors = G.neighbors(user_id)
list(neighbors)[:5]

['I-2', 'I-3', 'I-4', 'I-5', 'I-6']

Note que os vizinhos de um nó do tipo `item` são os são todos os usuários que o salvaram enquanto os vizinhos de um nó do tipo `usuário` são todos os itens salvos por este usuário.

Desta forma, se quisermos a construção "_Quem viu X também viu Y_" podemos seguir o caminho:

1. Escolha do item-alvo
2. Obtenção dos vizinhos do item-alvo (usuários que o salvaram)
3. Vizinhos dos usuários que viram o item-alvo

In [21]:
item_id = 'I-5448'
neighbors = G.neighbors(item_id)

neighbor_consumed_items = []
for user_id in neighbors:
    user_consumed_items = G.neighbors(user_id)
    neighbor_consumed_items += list(user_consumed_items)

print ('Quem viu {} também viu: {}'.format(item_id, ','.join(neighbor_consumed_items[:20])))

Quem viu I-5448 também viu: I-5186,I-4665,I-2937,I-1751,I-3010,I-5445,I-5446,I-5447,I-5444,I-5237,I-356,I-4139,I-4020,I-5449,I-3979,I-2991,I-4021,I-5450,I-3104,I-5448


Além disso, podemos fazer uma contagem dos itens que mais aparecem e utilizar este número como _score_ do item

In [22]:
consumed_items_count = Counter(neighbor_consumed_items)
consumed_items_count

Counter({'I-5186': 3,
         'I-4665': 1,
         'I-2937': 1,
         'I-1751': 4,
         'I-3010': 1,
         'I-5445': 1,
         'I-5446': 1,
         'I-5447': 1,
         'I-5444': 1,
         'I-5237': 2,
         'I-356': 3,
         'I-4139': 1,
         'I-4020': 1,
         'I-5449': 2,
         'I-3979': 1,
         'I-2991': 1,
         'I-4021': 2,
         'I-5450': 4,
         'I-3104': 3,
         'I-5448': 95,
         'I-5980': 1,
         'I-2706': 2,
         'I-4615': 2,
         'I-3966': 2,
         'I-2050': 1,
         'I-264': 1,
         'I-5981': 5,
         'I-5333': 1,
         'I-5973': 1,
         'I-5982': 1,
         'I-5983': 1,
         'I-3023': 4,
         'I-5804': 2,
         'I-3272': 1,
         'I-893': 1,
         'I-5984': 2,
         'I-832': 3,
         'I-2900': 2,
         'I-4995': 1,
         'I-1276': 1,
         'I-5941': 3,
         'I-79': 1,
         'I-425': 1,
         'I-4289': 1,
         'I-5968': 1,
         'I-6693

Por fim, podemos juntar a lógica na função `recommend_neighbor_items()` que recebe como parâmetris:
- `G`: o grafo de itens e usuários
- `target_id`: o ID do item-alvo
- `n`: o número de itens a serem recomendados

**Nota**: uma validação inicial é feita para validar se o ID do nó é do tipo `item`

In [33]:
def recommend_neighbor_items(G:nx.Graph, target_id, n=10):
    # Validando tipo do nó
    node_type = nx.get_node_attributes(G, 'node_type')[target_id]
    if node_type != 'item':
        raise ValueError('Node is not of item type.')

    # Analisando consumo dos usuários vizinhos
    neighbor_consumed_items = []
    for user_id in G.neighbors(target_id):
        user_consumed_items = G.neighbors(user_id)
        neighbor_consumed_items +=list(user_consumed_items)

    # Contabilizando itens consumidos pelos vizinhos
    consumed_items_count = Counter(neighbor_consumed_items)

    # Criando dataframe
    df_neighbors = pd.DataFrame(zip(consumed_items_count.keys(), consumed_items_count.values()))
    df_neighbors.columns = ['item_id', 'score']
    df_neighbors = df_neighbors.sort_values(by='score', ascending=False).set_index('item_id')
    return df_neighbors.head(n)

recommend_neighbor_items(G, 'I-5185', n=10)

,score
item_id,
I-5185,144
I-6618,28
I-3705,26
I-5942,24
I-5941,23
I-6080,23
I-4175,21
I-4003,21
I-6594,21


Note que o item-alvo é o que aparece em primeiro lugar na lista pois é a recomendação mais trivial ("_Quem viu X também viu X_"). No entanto, seu _score_ nos ajuda a analisar quantos vizinhos o item-alvo possui.

# Playground

Utilize as células abaixo para validar outras recomendações. Os itens populares também podem ser utilizados como base para a recomendação.

In [34]:
# Popular items
df.groupby('item_id')['user_id'].count().sort_values(ascending=False).head().index

Index(['I-487', 'I-442', 'I-773', 'I-3831', 'I-13'], dtype='object', name='item_id')

In [35]:
recommend_neighbor_items(G, 'I-487')

,score
item_id,
I-487,1544
I-773,170
I-817,153
I-931,150
I-877,139
I-495,130
I-1203,127
I-1198,127
I-485,125
